In [1]:
import os    

os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32"
import numpy as np
import pandas as pd
from scipy.misc import imread
import matplotlib.pyplot as plt
%matplotlib inline


import theano
from keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from keras.layers import Activation, Dropout, Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer, merge
# wow difference between Merge and merge---frustrating
from keras.models import Sequential, Model
from keras.optimizers import *
from keras.utils.np_utils import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score




Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GeForce 920M (CNMeM is disabled, cuDNN not available)
Using Theano backend.
/home/mrafi123/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
X_train=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/X_train.npy')
X_test=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/X_test.npy')
y_train=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/y_train.npy')
y_test=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/y_test.npy')

In [3]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [4]:
from keras.utils.np_utils import *

label_train=to_categorical(y_train)
label_test= to_categorical(y_test)
print (label_train.shape)
print (label_test.shape)


(50000, 10)
(10000, 10)


In [5]:
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print (x_train.shape)
print (x_test.shape)

(50000, 3072)
(10000, 3072)


In [6]:
x_train = x_train.reshape((len(x_train),3,32,32))
x_test = x_test.reshape((len(x_test),3,32,32))
print (x_train.shape)
print (x_test.shape)

(50000, 3, 32, 32)
(10000, 3, 32, 32)


In [7]:
img_rows, img_cols = 32, 32

nb_filters_1 = 96
nb_filters_2 = 192
nb_filters_3 = 256
nb_conv = 1
nb_conv_mid = 3
nb_conv_init = 5

In [8]:
def conv(init, nb_filter, row, col, subsample=(1,1), repeat=0):
    c = Convolution2D(nb_filter, row, col, border_mode='same', subsample=subsample)(init)
    c = LeakyReLU()(c)

    for i in range(repeat):
        c = Convolution2D(nb_filter, row, col, border_mode='same', subsample=subsample)(c)
        c = LeakyReLU()(c)
    return c

In [9]:
init = Input(shape=(3, img_rows, img_cols),)

fork11 = conv(init, nb_filters_1, nb_conv_init, nb_conv_init, repeat=1)
fork12 = conv(init, nb_filters_1, nb_conv_mid, nb_conv_mid, repeat=1)
merge1 = merge([fork11, fork12, ], mode='concat', concat_axis=1)
conv_pool1 = conv(merge1, nb_filters_1, nb_conv_init, nb_conv_init, subsample=(2,2))
bn1 = BatchNormalization(axis=1)(conv_pool1)

fork21 = conv(bn1, nb_filters_2, nb_conv_mid, nb_conv_mid, repeat=1)
fork22 = conv(bn1, nb_filters_2, nb_conv_mid, nb_conv_mid, repeat=1)
merge2 = merge([fork21, fork22, ], mode='concat', concat_axis=1)
conv_pool2 = conv(merge2, nb_filters_2, nb_conv_mid, nb_conv_mid, subsample=(2,2))
bn2 = BatchNormalization(axis=1)(conv_pool2)

fork31 = conv(bn2, nb_filters_3, nb_conv_mid, nb_conv_mid)
fork32 = conv(bn2, nb_filters_3, nb_conv, nb_conv)
fork33 = conv(bn2, nb_filters_3, nb_conv_init, nb_conv_init)
merge3 = merge([fork31, fork32, fork33, ], mode='concat', concat_axis=1)
conv_pool3 = conv(merge3, nb_filters_2, nb_conv_mid, nb_conv_mid, subsample=(2,2))
dropout = Dropout(0.5)(conv_pool3)

flatten = Flatten()(dropout)
output = Dense(10, activation="softmax")(flatten)

model = Model(input=init, output=output)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 32, 32)     0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 96, 32, 32)    7296        input_1[0][0]                    
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 96, 32, 32)    2688        input_1[0][0]                    
____________________________________________________________________________________________________
leakyrelu_1 (LeakyReLU)          (None, 96, 32, 32)    0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
        callbacks = [
                EarlyStopping(monitor='val_loss', patience=1, verbose=0),
                ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0),
        ]

In [11]:
        callbacks = [
                EarlyStopping(monitor='val_loss', patience=1, verbose=1)
        ]

In [12]:
model.fit(x_train, label_train,
            nb_epoch=3,
            batch_size=128,
            shuffle=True,
            validation_data=(x_test, label_test),
            callbacks=callbacks
            )

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 2226s - loss: 2.5686 - acc: 0.2478 - val_loss: 3.6936 - val_acc: 0.1832
Epoch 2/3
50000/50000 [==============================] - 2213s - loss: 2.5958 - acc: 0.2717 - val_loss: 3.1052 - val_acc: 0.2740
Epoch 3/3
50000/50000 [==============================] - 2222s - loss: 2.6973 - acc: 0.2848 - val_loss: 2.6438 - val_acc: 0.2881


In [64]:
y_pred = model.predict(x_test)
print (y_pred.size)
print (y_pred.shape)
print (y_pred.nbytes)

100000
(10000, 10)
800000


In [65]:
len(y_pred)

10000

In [66]:
y_pred[0]

array([ 0.08579691,  0.07687199,  0.12446834,  0.18162394,  0.10637974,
        0.12461416,  0.10059114,  0.09089096,  0.06253739,  0.04622545])

In [67]:
new_y_pred=[]
for i in range (len(y_pred)):
    new_y_pred.append([np.argmax(y_pred[i])])


In [68]:
new_y_pred=np.asarray(new_y_pred)

In [69]:
print (new_y_pred.shape)
new_y_pred=to_categorical(new_y_pred)
print (new_y_pred.shape)


(10000, 1)
(10000, 10)


In [70]:
new_y_pred

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [28]:
label_test

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.]])

In [71]:
num=len(y_pred)
r=0
w=0
for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(new_y_pred[i],label_test[i]):
            r+=1
        else:
            w+=1
print ("tested ",  num, "digits")
print ("correct: ", r, "wrong: ", w, "error rate: ", float(w)*100/(r+w), "%")
print ("got correctly ", float(r)*100/(r+w), "%")


tested  10000 digits
correct:  2749 wrong:  7251 error rate:  72.51 %
got correctly  27.49 %
